In [3]:
from win10toast import ToastNotifier

In [4]:
import time
import json
import urllib.request as finder
import datetime

In [5]:
# Authentication information for railways api
my_key="###########";

In [6]:
# This function interacts with railways API and extracts the relevant information regarding pnr status
# Input Args: pnr_number
# output : python dictonary containing information about current status of reservation
def get_data(pnr_number):
    try:
        url="https://api.railwayapi.com/v2/pnr-status/pnr/"+str(pnr_number)+"/apikey/"+str(my_key)+"/"
        data_response=finder.urlopen(url);
        data_files=data_response.read().decode('utf8')
        data_dict=json.loads(data_files)
        print("got_them")
        return data_dict
    except:
            print("Something wrong with your internet connection trying again after 60 sec")
            time.sleep(60)   
            get_data(pnr_number)

In [7]:
# This function  generates the pop up in your desktop regarding your updated pnr status
# input Args:message(string)
# Output: None
def get_notification(content):
    try:
        handle=ToastNotifier()
        handle.show_toast("Update on pnr status",content,icon_path=None,duration=30);
    except:
        print("Something went wrong we will trying again")
        get_notification(content)

In [13]:
# This is the main function which calls all other function and is responsible for running script
# Input Args:Pnr_number,interval(time interval after which u want to receive message)
# output Args:None
def main_func(pnr_number,interval):
    global exceptions
    data=get_data(pnr_number)
    if len(str(pnr_number))!=10:
        print("please input a valid 10 digit PNR number")
    elif data["response_code"]!=200:
            if data["response_code"]==210:
                print("Train doesn’t run on the date queried.");
            elif data["response_code"]==211:
                print("Train doesn’t have journey class queried.")
            elif data["response_code"]==220:
                print("Flushed PNR.")
            elif data["response_code"]==221:
                print("Invalid PNR.")
            elif data["response_code"]==230:
                print("Date chosen for the query is not valid for the chosen parameters")
            elif data["response_code"]==404:
                print("Data couldn’t be loaded on our servers. No data available.")
            elif data["response_code"]==405:
                print("Data couldn’t be loaded on our servers. Request couldn’t go through.")
            elif data["response_code"]==500:
                print("Unauthorized API Key.")
            elif data["response_code"]==501:
                print("Account Expired.")
            elif data["response_code"]==502:
                print("Invalid arguments passed.")
                      
    elif data["chart_prepared"]==0:
        print ("Chart not yet prepared please try after some time")
        get_notification("Chart not yet prepared please try after some time");
    else:
        i=1;
        archived_list=[];
            
        for passenger in data["passengers"]:
                database={};
                database["no"]=passenger["no"];
                database["current_status"]=passenger["current_status"]
                database["last_status"]=passenger["booking_status"]
                archived_list.append(database);
        
        while 1:
            time.sleep(interval) # sleeps the programe for the interval provided
            disp_info="";
            for (list_p) in (archived_list):
                if lsit_p["current_satus"]!=list_p["last_status"]: # if satus is changed saves the result to be displayd
                    str1="Update in reservation status of "+str(list_p["no"])+" current status is"+str(lsit_p["current_status"])+"\n"            
                    list_p["last_status"]=lsit_p["current_satus"]
                    disp_info+=str1;
            if len(disp_info)!=0:
                print("Status checked at "+ str(datetime.datetime.now()));
                for str1 in disp_info:
                    print(str1);
                    get_notification(str1);
                    time.sleep(30);
            else :
                get_notification("sorry no change in status since last time")
                
            data=get_data(pnr_number);
            for (p1,p2) in (data["passenger"],archived_list):
                p2["current_status"]=p1["current_status"];
            
            

In [12]:
# enter your input here
pnr_num=int(input("Enter your 10 Digit Pnr number"))
interval=int(input("Enter interval at which you want to check status"))
main_func(pnr_num,interval);


Enter your 10 Digit Pnr number2722699583
Enter interval at which you want to check status200
got_them
Chart not yet prepared please try after some time
